In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the MNIST dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)


# Define data loaders
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Load pre-trained model (ResNet18)
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)  # Replace the fully connected layer for 10 classes

# Allow the freezed pre-trained layers' parameters to be updated
for param in model.parameters():
    param.requires_grad = True

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [1]:
# Training loop
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    print(f'Train Accuracy: {(100 * correct / total):.2f}%')

# Evaluation on the test set
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate test accuracy
test_accuracy = (100 * correct / total)
test_loss /= len(test_loader)
print(f'Test Accuracy: {test_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}')

Epoch [1/20], Step [100/938], Loss: 0.6549
Epoch [1/20], Step [200/938], Loss: 0.3399
Epoch [1/20], Step [300/938], Loss: 0.3221
Epoch [1/20], Step [400/938], Loss: 0.2017
Epoch [1/20], Step [500/938], Loss: 0.2142
Epoch [1/20], Step [600/938], Loss: 0.2029
Epoch [1/20], Step [700/938], Loss: 0.2847
Epoch [1/20], Step [800/938], Loss: 0.2688
Epoch [1/20], Step [900/938], Loss: 0.1847
Train Accuracy: 90.48%
Epoch [2/20], Step [100/938], Loss: 0.2465
Epoch [2/20], Step [200/938], Loss: 0.1310
Epoch [2/20], Step [300/938], Loss: 0.1926
Epoch [2/20], Step [400/938], Loss: 0.1413
Epoch [2/20], Step [500/938], Loss: 0.2404
Epoch [2/20], Step [600/938], Loss: 0.2171
Epoch [2/20], Step [700/938], Loss: 0.1380
Epoch [2/20], Step [800/938], Loss: 0.2061
Epoch [2/20], Step [900/938], Loss: 0.1456
Train Accuracy: 95.13%
Epoch [3/20], Step [100/938], Loss: 0.1579
Epoch [3/20], Step [200/938], Loss: 0.2035
Epoch [3/20], Step [300/938], Loss: 0.1083
Epoch [3/20], Step [400/938], Loss: 0.1291
Epoch [3

Epoch [20/20], Step [900/938], Loss: 0.0610
Train Accuracy: 97.40%
Test Accuracy: 97.07%
Test Loss: 0.0927
